<div style="text-align: center; background-color: #5A96E3; font-family: 'Trebuchet MS', Arial, sans-serif; color: white; padding: 20px; font-size: 40px; font-weight: bold; border-radius: 0 0 0 0; box-shadow: 0px 6px 8px rgba(0, 0, 0, 0.2);">
  Stage 01 - Data collecting 📌
</div>

## I. Import libraries

In [ ]:
!pip install requests
!pip install pandas
!pip install beautifulsoup4
!pip install selenium
!pip install webdriver-manager

In [16]:
import os
import tempfile  # Make sure to import tempfile
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, WebDriverException
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import StaleElementReferenceException, NoSuchElementException
import time
import random
import pandas as pd
import requests
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager # Added import

## II. Config environment

In [17]:
CONFIG = {
    'url': 'https://topdev.vn/viec-lam-it',
    'login_url': 'https://accounts.topdev.vn/login/github',
    'login_github': 'https://github.com/login',
    'email': 'truongvietcong672@gmail.com',
    'password': 'Tienanh@12345',
    'max_pages': 1,
    'user_agents': [
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/134.0.0.0 Safari/537.36",
    ],
    'timeout': 650,
    'retry_attempts': 3,
    'retry_delay': 5,
}

## III. Collecting link html

### a. Set up drive

In [18]:
options = webdriver.ChromeOptions()
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('--headless=new') 
options.add_argument("window-size=1920,1080")

# User-agent ngẫu nhiên
user_agent = random.choice(CONFIG['user_agents'])
options.add_argument(f'user-agent={user_agent}')

driver = webdriver.Chrome(service=webdriver.chrome.service.Service(ChromeDriverManager().install()), options=options)

### b. Load page

In [19]:
# Login Github
driver.get(CONFIG['login_github'])
wait = WebDriverWait(driver, 3)

wait.until(EC.element_to_be_clickable((By.NAME, "login"))).send_keys(CONFIG['email'])
wait.until(EC.element_to_be_clickable((By.ID, "password"))).send_keys(CONFIG['password'])
wait.until(EC.element_to_be_clickable((By.NAME, "commit"))).click()

driver.get(CONFIG['login_url'])
print("Verifying login status on TopDev...")
time.sleep(3) # Give a moment for the page to settle after redirect

try:
    WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.XPATH, "//a[contains(@href, 'logout') or contains(@href, 'dang-xuat')] | //button[contains(translate(text(), 'ABCDEFGHIJKLMNOPQRSTUVWXYZ', 'abcdefghijklmnopqrstuvwxyz'), 'đăng xuất')] | //*[@aria-label='profile' or @aria-label='Profile' or @data-test-id='user-profile' or contains(@class, 'avatar') or contains(@class, 'profile')]"))
    )
    print("Login successful: Found an element indicating a logged-in state.")
except TimeoutException:
    print("Login verification failed: Could not find a clear indicator of a logged-in state within the timeout.")
    print(f"Current URL: {driver.current_url}")


driver.get(CONFIG['url'])

try:
    WebDriverWait(driver, 10).until(
        lambda d: d.execute_script("return document.readyState") == "complete"
    )
except Exception as e:
    print(f"An error occurred during the crawling process: {e}")

    # Get total records
num_record = int(
    driver.find_element(
        By.CSS_SELECTOR, "h1.text-xl.font-bold.text-black span.text-primary"
    ).text
)
print(f"Total records to fetch: {num_record}")

# Click to VIETNAM
driver.find_element(
    By.XPATH,
    "//button[contains(@class, 'uppercase') and contains(text(), 'Vi')]"
).click()

timeout = CONFIG['timeout']
start_time = time.time()
temp = 0

# Crolling to the bottom of the page
while True:
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)

    current_num_record = len(driver.find_elements(By.CSS_SELECTOR, ".mb-4.last\\:mb-0"))
    print(f"Current: {current_num_record}", end="\r")

    if current_num_record < temp:
        current_num_record = temp
        start_time = time.time()

    if current_num_record >= num_record:
        break

    if time.time() - start_time > timeout:
        print("Timeout: No change detected within the specified time.")
        break

    # temp = current_num_record

Verifying login status on TopDev...
Login successful: Found an element indicating a logged-in state.
Total records to fetch: 970
Timeout: No change detected within the specified time.


### c. Get job url

In [ ]:
url_list = []
elements = driver.find_elements(By.CSS_SELECTOR, ".mb-4.last\\:mb-0")

for i in range(len(elements)):
    try:
        # Refetch từng phần tử để tránh stale
        element = driver.find_elements(By.CSS_SELECTOR, ".mb-4.last\\:mb-0")[i]
        a_tag = element.find_element(By.TAG_NAME, "a")
        href = a_tag.get_attribute("href")
        if href:
            url_list.append(href)
    except (StaleElementReferenceException, NoSuchElementException):
        print(f"Bỏ qua phần tử lỗi tại index {i}")
        continue
now = pd.to_datetime("now")
print(f"\nCrawling completed at {now}. Total URLs collected: {len(url_list)}")

for i, url in enumerate(url_list):
    print(f"{i + 1}/{len(url_list)}: {url}")

# Save URLs to CSV
df = pd.DataFrame(url_list, columns=["url"])
df.to_csv('../data/job_url.csv', index=False)

Bỏ qua phần tử lỗi tại index 5

Crawling completed at 2025-05-12 18:58:12.989708. Total URLs collected: 950
1/950: https://topdev.vn/viec-lam/dba-cong-ty-tnhh-samsung-sds-viet-nam-2036332?src=topdev_search&medium=searchresult
2/950: https://topdev.vn/viec-lam/nhan-vien-kinh-doanh-b2b-ai-giao-duc-ngoai-ngu-co-the-giao-tiep-tieng-han-cong-ty-tnhh-weavers-vietnam-2036330?src=topdev_search&medium=searchresult
3/950: https://topdev.vn/viec-lam/project-manager-playable-ads-team-52-entertainment-2036305?src=topdev_search&medium=searchresult
4/950: https://topdev.vn/viec-lam/senior-unity-developer-playable-ads-52-entertainment-2036304?src=topdev_search&medium=searchresult
5/950: https://topdev.vn/viec-lam/junior-unity-developer-playable-ads-52-entertainment-2036303?src=topdev_search&medium=searchresult
6/950: https://topdev.vn/viec-lam/middle-senior-level-ky-su-phat-trien-fullstack-java-fullstack-developer-khoi-cong-nghe-thong-tin-holt-12-mbbank-2036170?src=topdev_search&medium=searchresult
7/

## IV. Crawl data

Ta sẽ chọn ra các trường thông tin sau:
- `job_title`: tên công việc (VD: IT Helpdesk, Data engineer, ...vv)

- `company_name`: tên công ty (VD: Vingroup, Momo, ...vv)

- `salary`: mức lương (VD: 2.000.000vnđ, 15.000.000vnđ..vv)

- `address`: địa chỉ (VD: Tân Bình, Bình thạnh...vv)

- `posted_date`: ngày đăng (VD: 21/6/2025, 21/7/2025)

- `industry`: ngành nghề (VD: Giáo dục, Phần mềm, ...vv)

- `company_size`: quy mô công ty (VD: 500-999 Nhân viên)

- `company_nationality`: quốc tịch công ty (VD: China, India, United-Kingdom, Vietnam)

- `years_experience`: năm kinh nghiệm (VD: từ 2 năm, ...vv)

- `position_level`: cấp bậc (VD: Intern, Middle, Senior)

- `employment_type`: loại hình (VD: In Office, Remote..vv)

- `contract_type`: loại hợp đồng (VD: Fulltime, Freelance)

- `technologies_used`: công nghệ sử dụng (VD: .NET, Java)

- `job_description`: mô tả công việc (VD: Trách nhiệm công việc là Làm việc với các bộ ph.)

- `company_url`: url thông tin công ty.

- `url`: url công việc

In [21]:
data = []

for i, url in enumerate(url_list):
    driver.get(url)
    WebDriverWait(driver, 10).until(
        lambda d: d.execute_script("return document.readyState") == "complete"
    )
    time.sleep(2)

    # Tên công việc
    job_title_ele = driver.find_elements(
        By.CSS_SELECTOR, 
        "h1.text-2xl.font-bold.text-black"
    )
    job_title = job_title_ele[0].text.strip() if job_title_ele else None

    # Tên công ty
    company_name_ele = driver.find_elements(
        By.CSS_SELECTOR, 
        "p.text-base.font-bold"
    )
    company_name = company_name_ele[0].text.strip() if company_name_ele else None

    # Mức lương
    salary_elem = driver.find_elements(
        By.CSS_SELECTOR, 
        "div.flex.items-center p.text-primary"
    )
    salary = salary_elem[0].text.strip() if salary_elem else None

    # Địa chỉ
    address_elem = driver.find_elements(
        By.CSS_SELECTOR, 
        "div[data-testid='flowbite-tooltip-target'] span"
    )
    address = [elem.text.strip() for elem in address_elem if elem.text.strip()]

    # Ngày đăng
    date_posted_elem = driver.find_elements(
        By.XPATH, 
        "//div[contains(text(), 'Đăng')]"
    )
    date_posted = [elem.text.split("và")[0].strip() for elem in date_posted_elem if elem.text.startswith("Đăng")]

    # Ngành nghề
    industry_elem = driver.find_elements(
        By.XPATH, 
        "//h3[text()='Ngành nghề']/following-sibling::p"
    )
    industry = [elem.text.strip() for elem in industry_elem if elem.text.strip()]

    # Quy mô công ty
    company_size_elem = driver.find_elements(
        By.XPATH, 
        "//h3[text()='Quy mô công ty']/following-sibling::p"
    )
    company_size = [elem.text.strip() for elem in company_size_elem if elem.text.strip()]

    # Quốc tịch công ty
    company_nationality_elem = driver.find_elements(
        By.XPATH, 
        "//h3[text()='Quốc tịch công ty']/following-sibling::p"
    )
    company_nationality = [elem.text.strip() for elem in company_nationality_elem if elem.text.strip()]

    # Năm kinh nghiệm
    experience_years_elem = driver.find_elements(
        By.XPATH, 
        "//h3[text()='Năm kinh nghiệm tối thiểu']/following-sibling::div//a"
    )
    experience_years = [elem.text.strip() for elem in experience_years_elem if elem.text.strip()]

    # Cấp bậc
    position_level_elem = driver.find_elements(
        By.XPATH, 
        "//h3[text()='Cấp bậc']/following-sibling::div//a"
    )
    position_level = [elem.text.strip() for elem in position_level_elem if elem.text.strip()]

    # Loại hình
    employment_type_elem = driver.find_elements(
        By.XPATH, 
        "//h3[text()='Loại hình']/following-sibling::div//a"
    )
    employment_type = [elem.text.strip() for elem in employment_type_elem if elem.text.strip()]

    # Hợp đồng
    contract_type_elem = driver.find_elements(
        By.XPATH, 
        "//h3[text()='Loại hợp đồng']/following-sibling::div//a"
    )
    contract_type = [elem.text.strip() for elem in contract_type_elem if elem.text.strip()]

    # Công nghệ sử dụng
    technologies_used_elem = driver.find_elements(
        By.XPATH, 
        "//div[contains(@class, 'flex flex-wrap')]//a/span"
    )
    technologies_used = [elem.text.strip() for elem in technologies_used_elem if elem.text.strip() and elem.text.strip() != 'Xem công ty']

    # Mô tả công việc
    job_description_elem = driver.find_elements(
        By.ID, 
        "JobDescription"
    )
    job_description = [elem.text.strip() for elem in job_description_elem if elem.text.strip()]

    # URL Công ty
    company_url_elem = driver.find_elements(
        By.XPATH, 
        "//a[contains(@class, 'break-all') and contains(@href, 'topdev.vn/nha-tuyen-dung')]"
    )
    company_url = [elem.get_attribute("href").strip() for elem in company_url_elem if elem.get_attribute("href")]

    # Lưu trữ dữ liệu
    job = [
        job_title,
        company_name,
        salary,
        address,
        date_posted,
        industry,
        company_size,
        company_nationality,
        experience_years,
        position_level,
        employment_type,
        contract_type,
        technologies_used,
        job_description,
        company_url,
        url
    ]
    data.append(job)

    print(f"Data was collected from {i + 1}/{len(url_list)}: {url}")

print("Data collection completed.")
print(f"Total records collected: {len(data)}")

Data was collected from 1/950: https://topdev.vn/viec-lam/dba-cong-ty-tnhh-samsung-sds-viet-nam-2036332?src=topdev_search&medium=searchresult
Data was collected from 2/950: https://topdev.vn/viec-lam/nhan-vien-kinh-doanh-b2b-ai-giao-duc-ngoai-ngu-co-the-giao-tiep-tieng-han-cong-ty-tnhh-weavers-vietnam-2036330?src=topdev_search&medium=searchresult
Data was collected from 3/950: https://topdev.vn/viec-lam/project-manager-playable-ads-team-52-entertainment-2036305?src=topdev_search&medium=searchresult
Data was collected from 4/950: https://topdev.vn/viec-lam/senior-unity-developer-playable-ads-52-entertainment-2036304?src=topdev_search&medium=searchresult
Data was collected from 5/950: https://topdev.vn/viec-lam/junior-unity-developer-playable-ads-52-entertainment-2036303?src=topdev_search&medium=searchresult
Data was collected from 6/950: https://topdev.vn/viec-lam/middle-senior-level-ky-su-phat-trien-fullstack-java-fullstack-developer-khoi-cong-nghe-thong-tin-holt-12-mbbank-2036170?src=

In [ ]:
collumns = [
    "job_title",
    "company_name",
    "salary",
    "address",
    "date_posted",
    "industry",
    "company_size",
    "company_nationality",
    "experience_years",
    "position_level",
    "employment_type",
    "contract_type",
    "technologies_used",
    "job_description",
    "company_url",
    "job_url"
]

df = pd.DataFrame(data, columns=collumns)

# Update address from link companies_url
new_address = []
count = 1

for url in df["company_url"]:
    if not url:
        new_address.append([])
        continue
    url_to_visit = url[0] if isinstance(url, list) else url

    print(f"Updating addresses from {count}/{len(df)}: {url_to_visit}")
    driver.get(url_to_visit)
    time.sleep(2)  # Đợi trang tải xong

    try:
        # Lấy tất cả địa chỉ (nhiều <p> trong <ul>)
        address_elems = driver.find_elements(
            By.XPATH, 
            "//h3[text()='Địa chỉ công ty']/following-sibling::ul//p"
        )
        addresses = [elem.text.strip() for elem in address_elems if elem.text.strip()]
        
        print(f"Update addresses successfully from {count}/{len(df)}: {url}")
        count += 1
    except Exception as e:
        addresses = []
        print(f"Lỗi khi lấy địa chỉ từ {url}: {e}")

    new_address.append(addresses)

df['address'] = new_address
driver.quit()
df.head()

,job_title,company_name,salary,address,date_posted,industry,company_size,company_nationality,experience_years,position_level,employment_type,contract_type,technologies_used,job_description,company_url,job_url
0,DBA,CÔNG TY TNHH SAMSUNG SDS VIỆT NAM,Thương lượng,"[Quận Cầu Giấy, Hà Nội]",[Đăng 7 giờ trước],[Dịch vụ IT],[Hơn 1000 Nhân viên],[South-Korea],[Từ 5 năm],"[Middle, Senior]",[In Office],[Fulltime],"[Oracle Rac/ SQL Server, Database Administrato...",[SALARY\nNegotiable\nRESPONSIBILITIES\nCoordin...,[https://topdev.vn/nha-tuyen-dung/cong-ty-tnhh...,https://topdev.vn/viec-lam/dba-cong-ty-tnhh-sa...
1,Nhân Viên Kinh Doanh B2B - AI Giáo Dục Ngoại N...,CÔNG TY TNHH WEAVERS VIETNAM,18.000.000 VND to 20.000.000 VND,"[Quận Nam Từ Liêm, Hà Nội]",[Đăng 8 giờ trước],"[Giáo dục, EdTech, Artificial Intelligence]",[25-99 Nhân viên],[South-Korea],[Từ 1 năm],[Junior],[In Office],[Fulltime],"[IT Consultant, AI, Korean]",[SALARY\n18.000.000 VND to 20.000.000 VND\nRES...,[https://topdev.vn/nha-tuyen-dung/cong-ty-tnhh...,https://topdev.vn/viec-lam/nhan-vien-kinh-doan...
2,Project Manager (Playable Ads Team),52 Entertainment,Thương lượng,"[Quận 1, Hồ Chí Minh]",[Đăng 10 giờ trước],"[Phần Mềm, Giải trí/ Game]",[10-24 Nhân viên],[Vietnam],[Từ 2 năm],"[Middle, Senior]",[In Office],[Fulltime],[Project Manager],[We're looking for a proactive and detail-orie...,[https://topdev.vn/nha-tuyen-dung/52-entertain...,https://topdev.vn/viec-lam/project-manager-pla...
3,Senior Unity Developer (Playable Ads),52 Entertainment,Thương lượng,"[Quận 1, Hồ Chí Minh]",[Đăng 10 giờ trước],"[Phần Mềm, Giải trí/ Game]",[10-24 Nhân viên],[Vietnam],[Từ 3 năm],[Senior],[In Office],[Fulltime],"[Unity, C#, Git]",[We’re seeking an experienced and driven Senio...,[https://topdev.vn/nha-tuyen-dung/52-entertain...,https://topdev.vn/viec-lam/senior-unity-develo...
4,Junior Unity Developer (Playable Ads),52 Entertainment,Thương lượng,"[Quận 1, Hồ Chí Minh]",[Đăng 10 giờ trước],"[Phần Mềm, Giải trí/ Game]",[10-24 Nhân viên],[Vietnam],[Từ 1 năm],[Junior],[In Office],[Fulltime],"[Unity, C#, Git]",[We’re looking for a motivated and detail-orie...,[https://topdev.vn/nha-tuyen-dung/52-entertain...,https://topdev.vn/viec-lam/junior-unity-develo...


### V. Save data

In [28]:
df.to_csv('../data/raw_data.csv', index=False)
print("Data saved to data.csv")

Data saved to data.csv
